# Clustering the neighborhoods in Toronto

### Import necessary Libraries

In [13]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Download and Explore Dataset

In [14]:
toronto_data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0) # read all tables from website
#print(toronto_data)

In [15]:
toronto_data[0].head(10) # explore the table of interest

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [16]:
postal_code = toronto_data[0]
postal_code = postal_code[postal_code.Borough != 'Not assigned'].reset_index(drop=True) # Ignore cells with a borough that is 'Not assigned'
print(postal_code.shape)
postal_code.head(10)

(211, 3)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [17]:
#postal_code = postal_code[postal_code.Borough != 'Not assigned']

# group by Postcode and Borough to get a column 'Neighbourhood' which contains all neighbourhoods
postal_code_dense = postal_code.groupby(['Postcode','Borough']).Neighbourhood.agg([('Neighbourhood', ', '.join)]).reset_index()

In [18]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
ind = postal_code_dense.Neighbourhood == 'Not assigned'
postal_code_dense.loc[ind,'Neighbourhood'] = postal_code_dense.loc[ind,'Borough']

postal_code_dense.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [19]:
postal_code_dense.shape # print shape of dataframe

(103, 3)